In [73]:
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma
from valentine.algorithms import Cupid
import os
import multiprocessing as mp
from multiprocessing import Pool
import time
import pandas as pd
from compute_match import *

# Preparazione data frames

In [74]:
csv_path = './csv/'

## Creazione dizionari che associa il nome del file al file .json

In [75]:
# dataset sono in formato json necessariamente
path = "../../json_datasets"

# Dizionario Key=Nome del file, Value=File json associato (letto da filepath)
data_frames = {}
# Per tutti i file dentro la cartella dei dataset
for file in os.listdir(path):
    # Prendo il path del file
    filepath = f"{path}/{file}"
    # Update del dizionario
    data_frames[file] = (pd.read_json(filepath))

## Creazione di tutte le coppie di dataset da valutare (evitando ripetizioni)

In [76]:
# Prendo tutti i nomi dei file
dbFileNames = list(data_frames.keys())
# Tuple = (nome dataset1, nome dataset2, dataframe1, dataframe2)
allDbsCouples = []
# Il numero di dataframes
dfs_len = len(dbFileNames)

In [77]:
# Preso un dataframe, lo confronto con tutti i dataframes successivi (così evito di confrontare coppie già confrontate)
for i in range(dfs_len):
    for j in range(i + 1, dfs_len):
        dfl = data_frames[dbFileNames[i]]
        dfr = data_frames[dbFileNames[j]]
        # le tuple da passare al metodo di matching = nome dataset1, nome dataset2, dataframe1, dataframe2
        allDbsCouples.append((dbFileNames[i], dbFileNames[j], dfl, dfr))

# Valentine schema matching

In [78]:
def calculate_match_multithread(tuples):
    result = []
    # Creo un pool per l'esecuzione multi processore (per velocizzare il matching tra tutte le coppie di dataframes)
    with Pool() as pool:
        # Ogni pool esegue la funzione (primo parametro), passandogli le tuple (secondo parametro)
        # result è una lista di tuple ove una tupla è il risultato della funzione mappata
        result = pool.map(calculate_match_coma_schema, tuples)
    return result

def matches_to_table(result):
    matches = pd.DataFrame()
    # Per ogni risultato ottenuto dal matching
    for match in result:
    # Prendo il valore ritornato dalla funzione valentine_match 
    # e.g. match[2] = ((table_1, 'Cited by'),(table_2, 'Cited by')): 0.83
        for key in match[2].keys():
            # e.g. key = (table_1, 'Cited by'),(table_2, 'Cited by')
            d = dict()
            d["table_1"] = match[0]
            d["table_2"] = match[1]
            d["column_table_1"] = key[0][1]
            d["column_table_2"] = key[1][1]
            d["match_value"] = match[2][key]
            # Aggiungiamo al dataframe l'ennupla
            matches = pd.concat([matches, pd.DataFrame(d, index=[0])], ignore_index=True)
    return matches

# Tabella per la visualizzazione dei risultati dello schema matching con Valentine

In [79]:
# start_time = time.perf_counter()
# # Calcola i match per tutte le coppie di datasets definite in tuples
# result = calculate_match_multithread(allDbsCouples)
# finish_time = time.perf_counter()

# # Definizione del nome delle colonne della tabella
# columns = ["table_1", "table_2", "column_table_1", "column_table_2", "match_value"]
# # Creo il dataframe (la tabella in questione)
# matches = pd.DataFrame(columns=columns)

# # Porta tutti i match in formato tabellare
# matches = matches_to_table(result)

In [80]:
# pd.DataFrame(matches).to_csv(csv_path + "matches.csv")

# Schema mediato

In [81]:
# Schema mediato definito manualmente tramite attributi di interesse e che fanno maggior match
mediated_schema_columns = ['name', 'country', 'market cap', 'founded year', 'employees', 'industry', 'sector',
     'ceo', 'revenue', 'stock', 'share price', 'city', 'address', 'website']


In [82]:
mediated_schema = pd.DataFrame(columns=mediated_schema_columns)

In [83]:
# Creazione di tutte le coppie (schema-mediato, dataset)
matches_with_mediated = {}
all_mediatedNDbs_couples = []
for i in range(dfs_len):
    dfl = mediated_schema
    dfr = data_frames[dbFileNames[i]]
    # le tuple da passare al metodo di matching = nome dataset1, nome dataset2, dataframe1, dataframe2
    all_mediatedNDbs_couples.append(('mediated_schema', dbFileNames[i], dfl, dfr))

In [84]:
# Matches con lo schema mediato
result = calculate_match_multithread(all_mediatedNDbs_couples)

In [85]:
matches = matches_to_table(result)

In [86]:
pd.DataFrame(matches).to_csv(csv_path + "matches_with_mediated.csv")

# Riempimento schema mediato

In [87]:
matches = pd.read_csv(csv_path + 'matches_with_mediated.csv')
tables_to_columns = dict()
# Per tutte le tabelle in match nella colonna table 2 vado a prendermi
# le colonne con match >= 0.5 con una colonna dello schema mediato
for table_name in set(matches['table_2']):
    # Prende le colonne con table 2 uguale a table name
    table = matches.loc[matches['table_2'] == table_name]
    columns = set()
    for ind in table.index:
        if table['match_value'][ind] >= 0.5:
            columns.add((table['column_table_1'][ind], table['column_table_2'][ind]))
    tables_to_columns[table_name] = columns

## Risoluzione problemi con colonne duplicate

In [88]:
import regex as re
schema_final = pd.DataFrame(columns=mediated_schema_columns)

# Per tutti i dataset vai a inserire i dati nello schema mediato
for df_name in tables_to_columns.keys():
    df_aligned = pd.DataFrame()
    key2maxYear = {}
    colsToRemove = []

    for match in tables_to_columns[df_name]:

        if match[0] not in key2maxYear:
            if re.findall("[0-9]{4}", match[1]) != []:
                key2maxYear[match[0]] = re.findall("[0-9]{4}", match[1])[0], match[1]
            else:
                key2maxYear[match[0]] = 0, match[1]
        else:
            currYear = re.findall("[0-9]{4}", match[1])[0]

            if key2maxYear[match[0]][0] < currYear:
                key2maxYear[match[0]] = currYear, match[1]
                
    # Per tutti i match del dataset effettua una proiezione sulle colonne in match e rinominale per allinearle allo schema mediato
    for mediated_column in key2maxYear:
        tmp = data_frames[df_name][key2maxYear[mediated_column][1]].to_frame()
        tmp = tmp.rename(columns={key2maxYear[mediated_column][1]: mediated_column})

        df_aligned = pd.concat([df_aligned, tmp], axis=1)
    
    try:
        schema_final = pd.concat([schema_final, df_aligned], ignore_index=True, sort=False)
    except:
        print("Cannot concat schema final with: ", df_name)

In [89]:
schema_final.to_csv(csv_path + "schema_final.csv", index_label='id')

# Dataset con prime 10 ennuple (usate per magellan su colab)

In [90]:
# sch = schema_final.iloc[:10]
# sch.to_csv(csv_path + "schema_final_10.csv", index_label='id')